## Webscrapping from metroscubicos.com

In [ ]:
# import instances and libraries
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import requests
import re
import pandas as pd
# To show a progess bar in loops:
from tqdm import tqdm_notebook as tqdm
import csv

In [ ]:
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
# List all entities available in metroscubicos.com

entidades = ['Aguascalientes', 'Baja California', 'Baja California Sur', 'Campeche', 'Chiapas', 'Chihuahua', 'Coahuila', 'Colima',
            'Distrito Federal', 'Durango', 'Estado De México', 'Guanajuato', 'Guerrero', 'Hidalgo', 'Jalisco', 'Michoacán',
             'Morelos', 'Nayarit', 'Nuevo León', 'Oaxaca', 'Puebla', 'Querétaro', 'Quintana Roo', 'San Luis Potosí',
             'Sinaloa', 'Sonora', 'Tabasco', 'Tamaulipas', 'Tlaxcala', 'Veracruz', 'Yucatán', 'Zacatecas']

In [ ]:
# make smallcaps and change spaces for dashes, and select which entities
entidadesLower = [x.lower().replace(' ', '-') for x in entidades]

In [ ]:
entidadesLower

In [ ]:
# Initialize variables
cityList = []
listLinks = []
tipos = ['casas', 'departamentos', 'terrenos']
baseUrl = 'https://inmuebles.metroscubicos.com/'
csvPath = './Resources/'

### Script para obtener links y guardarlos en csv por ciudad

In [ ]:
# PRIMERO OBTENER TODOS LOS LINKS

# Loop para navegar en cada ciudad
for ciudad in tqdm(entidadesLower):
    # initialize counter for ciudad
    counter = 0
    links = []
    #cityDict = {}
    #browser = Browser('chrome', **executable_path, headless=False)
    for tipo in tqdm(tipos):
        browseUrl = baseUrl+tipo+"/venta/"+ciudad
        # visit results for ciudad
        browser.visit(browseUrl)
        html = browser.html
        html_soup = soup(html, 'html.parser')
        # Get number of pages for results
        paginas = html_soup.find('li', class_=re.compile('page-count'))
        if paginas != None:
            numPaginas = [int(s) for s in paginas.get_text().split() if s.isdigit()][0]
        else:
            numPaginas = 1
        if browser.is_element_present_by_id('newCookieDisclaimerButton'):
            browser.find_by_css('button[id="newCookieDisclaimerButton"]').click()
        
        

        # Browse in each page
        for pagina in tqdm(range(numPaginas)):
            # Get all links
            htmlNew = browser.html
            new_soup = soup(htmlNew, 'html.parser')
            resultados = new_soup.find_all('a', class_="ui-search-link")
            link=""
            for resultado in resultados:
                cityDict = {}
                if "MLM-" in resultado['href']:
                    if resultado['href'] != "" and re.search('[^#]*',resultado['href']) != None:
                        link = re.search('[^#]*',resultado['href']).group(0)
                    else:
                        continue
                else:
                    continue
                if re.search('(?<=MLM-)\d+',link) != None:
                    ID = re.search('(?<=MLM-)\d+',link).group(0)
                else:
                    continue
                if re.search('(?:MLM-\d+-)(\w+)',link) != None:
                    tipo = re.search('(?:MLM-\d+-)(\w+)',link)[1]
                else:
                    tipo = ""
                cityDict["ID"] = ID
                cityDict["entidad"] = ciudad
                cityDict["tipo"] = tipo
                cityDict["link"] = link
                if cityDict not in links:
                    links.append(cityDict)

            # Change page if hasn't reach final
            if (pagina+1) != numPaginas and browser.find_by_css('li[class="andes-pagination__button andes-pagination__button--next"] a') != None:
                browser.find_by_css('li[class="andes-pagination__button andes-pagination__button--next"] a').first.click()
            
    # save links to csv per city
    file = csvPath+ciudad+".csv"
    df = pd.DataFrame(links)
    df.to_csv(file, index=False)
    print(f'Total links for {ciudad}:{len(links)}')    

browser.quit()

